In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12310286813151707989
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4002552351749811426
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13384218085320206659
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10770630592
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8509874950062444451
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [3]:
cd /media/datastorage/Phong/

/media/datastorage/Phong


In [4]:
ls -l

total 7191252
drwxrwxr-x 20 bribeiro bribeiro       4096 ago 23  2019 cassava/
drwxrwxr-x  5 bribeiro bribeiro       4096 mar 18  2021 cifar10/
drwxrwxr-x 15 bribeiro bribeiro       4096 set  3  2020 cifar100_png/
-rw-rw-r--  1 bribeiro bribeiro  169001437 ago  8  2020 cifar-100-python.tar.gz
drwxr-xr-x  2 bribeiro bribeiro       4096 fev  6  2021 dataset5/
drwxrwxr-x 15 bribeiro bribeiro       4096 set  3  2019 DogAge/
drwxrwxr-x 17 bribeiro bribeiro       4096 ago  8  2020 fashion_mnist/
-rw-rw-r--  1 bribeiro bribeiro 2246539027 fev  4  2021 fingerspelling5.tar.bz2
drwxrwxr-x  6 bribeiro bribeiro       4096 ago  5  2019 ImageNet/
drwxrwxr-x  3 bribeiro bribeiro       4096 fev 24  2021 ImageNet2/
-rw-rw-r--  1 bribeiro bribeiro  793579520 out 20  2011 images.tar
-rw-rw-r--  1 bribeiro bribeiro  182349464 set 22  2019 libcudnn7_7.6.4.38-1+cuda10.1_amd64.deb
-rw-rw-r--  1 bribeiro bribeiro  160754004 set 22  2019 libcudnn7-dev_7.6.4.38-1+cuda10.1_amd64.deb
drwxrwxr-x  3 bribeiro bribei

In [5]:
# #Images/n02105855-Shetland_sheepdog/n02105855_9415.jpg

# import glob
# import os

# import numpy as np
# from matplotlib.image import imread
# %matplotlib inline
# import matplotlib.pyplot as plt

# # get image parts
# def get_image_parts(image_path):
#     """Given a full path to an image, return its parts."""
#     parts = image_path.split(os.path.sep)
#     #print(parts)
#     filename = parts[2]
#     filename_no_ext = filename.split('.')[0]
#     classname = parts[1]
#     train_or_test = parts[0]
    
#     return train_or_test, classname, filename_no_ext, filename
    
    
# sample_images = list(glob.glob(os.path.join('train/', '*/*'), recursive=True))
# np.random.seed(42)
# rand_imgs = np.random.choice(sample_images, size=5*5)
# fig, axarr = plt.subplots(5, 5, figsize=(20, 20))

# for i, rand_img in enumerate(rand_imgs):
#     train_or_test, classname, filename_no_ext, filename = get_image_parts(rand_img)
    
#     j = i // 5
#     k = i % 5
#     axarr[j][k].imshow(imread(rand_img))
#     axarr[j][k].title.set_text(classname)
#     axarr[j][k].grid(False)

In [6]:
from keras.callbacks import Callback
import pickle
import sys

#Stop training on val_acc
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

#Save large model using pickle formate instead of h5            
class SaveCheckPoint(Callback):
    def __init__(self, model, dest_folder):
        super(Callback, self).__init__()
        self.model = model
        self.dest_folder = dest_folder
        
        #initiate
        self.best_val_acc = 0
        self.best_val_loss = sys.maxsize #get max value
          
    def on_epoch_end(self, epoch, logs={}):
        val_acc = logs['val_acc']
        val_loss = logs['val_loss']

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            
            # Save weights in pickle format instead of h5
            print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
            weigh= self.model.get_weights()

            #now, use pickle to save your model weights, instead of .h5
            #for heavy model architectures, .h5 file is unsupported.
            fpkl= open(self.dest_folder, 'wb') #Python 3
            pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
            fpkl.close()
            
#             model.save('tmp.h5')
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                self.best_val_loss=val_loss
                
                # Save weights in pickle format instead of h5
                print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
                weigh= self.model.get_weights()

                #now, use pickle to save your model weights, instead of .h5
                #for heavy model architectures, .h5 file is unsupported.
                fpkl= open(self.dest_folder, 'wb') #Python 3
                pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
                fpkl.close()                    

In [7]:
# !pip3 install -U git+https://github.com/qubvel/efficientnet

In [8]:
#MUL 1 - Inception - ST

from keras.applications import InceptionV3
# from keras.applications import Xception
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.applications import EfficientNetB0

from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, SimpleRNN, LSTM, Flatten, GRU, Reshape

from keras.applications.inception_v3 import preprocess_input
# from tensorflow.keras.applications.efficientnet import preprocess_input

from keras.layers import GaussianNoise

def get_adv_model():
    f1_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3))  
    
#     #frozen layers    
#     for layer in f1_base.layers:
#         layer.trainable = False  
        
    f1_x = f1_base.output
    f1_x = GlobalAveragePooling2D()(f1_x)    
    
#     f1_base = EfficientNetB0(include_top=False, weights='imagenet', 
#                     input_shape=(299, 299, 3), 
#                     pooling='avg')
#     f1_x = f1_base.output

# f1_x = f1_base.layers[-151].output   #layer 5

# f1_x = GlobalAveragePooling2D()(f1_x)
# f1_x = Flatten()(f1_x)

# f1_x = Reshape([1,1280])(f1_x)  
# f1_x = SimpleRNN(2048, 
#             return_sequences=False,                       
# #             dropout=0.8                                     
#             input_shape=[1,1280])(f1_x)
   
    #Regularization with noise
    f1_x = GaussianNoise(0.1)(f1_x)

    f1_x = Dense(1024, activation='relu')(f1_x)
    f1_x = Dense(10, activation='softmax')(f1_x)
    model_1 = Model(inputs=[f1_base.input],outputs=[f1_x])
    model_1.summary()
    
    return model_1


In [9]:
# model_mul_test = get_adv_model()
# for layer in model_mul_test.layers:
#     print(layer.name, ': ', layer.trainable)

In [ ]:
# import numpy as np
# import random

# class CutMixImageDataGenerator():
#     def __init__(self, generator1, generator2, img_size, batch_size):
#         self.batch_index = 0
#         self.samples = generator1.samples
#         self.class_indices = generator1.class_indices
#         self.generator1 = generator1
#         self.generator2 = generator2
#         self.img_size = img_size
#         self.batch_size = batch_size

#     def reset_index(self):  # Ordering Reset (If Shuffle is True, Shuffle Again)
#         self.generator1._set_index_array()
#         self.generator2._set_index_array()

#     def reset(self):
#         self.batch_index = 0
#         self.generator1.reset()
#         self.generator2.reset()
#         self.reset_index()

#     def get_steps_per_epoch(self):
#         quotient, remainder = divmod(self.samples, self.batch_size)
#         return (quotient + 1) if remainder else quotient
    
#     def __len__(self):
#         self.get_steps_per_epoch()

#     def __next__(self):
#         if self.batch_index == 0: self.reset()

#         crt_idx = self.batch_index * self.batch_size
#         if self.samples > crt_idx + self.batch_size:
#             self.batch_index += 1
#         else:  # If current index over number of samples
#             self.batch_index = 0

#         reshape_size = self.batch_size
#         last_step_start_idx = (self.get_steps_per_epoch()-1) * self.batch_size
#         if crt_idx == last_step_start_idx:
#             reshape_size = self.samples - last_step_start_idx
            
#         X_1, y_1 = self.generator1.next()
#         X_2, y_2 = self.generator2.next()
        
#         cut_ratio = np.random.beta(a=1, b=1, size=reshape_size)
#         cut_ratio = np.clip(cut_ratio, 0.2, 0.8)
#         label_ratio = cut_ratio.reshape(reshape_size, 1)
#         cut_img = X_2

#         X = X_1
#         for i in range(reshape_size):
#             cut_size = int((self.img_size-1) * cut_ratio[i])
#             y1 = random.randint(0, (self.img_size-1) - cut_size)
#             x1 = random.randint(0, (self.img_size-1) - cut_size)
#             y2 = y1 + cut_size
#             x2 = x1 + cut_size
#             cut_arr = cut_img[i][y1:y2, x1:x2]
#             cutmix_img = X_1[i]
#             cutmix_img[y1:y2, x1:x2] = cut_arr
#             X[i] = cutmix_img
            
#         # X = seq.augment_images(X)  # Sequential of imgaug
#         y = y_1 * (1 - (label_ratio ** 2)) + y_2 * (label_ratio ** 2)
#         return X, y

#     def __iter__(self):
#         while True:
#             yield next(self)

In [15]:
# Training
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam, SGD

import time, os
from math import ceil
import random

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
#     horizontal_flip=True,
#     vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    # preprocessing_function=get_cutout_v2(),
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 1
batch_size = 32

train_set = train_datagen.flow_from_directory('cifar10/cifar10v3/train_resized_299/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('cifar10/cifar10v3/test_resized_299/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'st'
# Helper: Save the model.
savedfilename = os.path.join('cifar10', 'cifar10v3', 'checkpoints', 'Cifar10_Share_1_Inception_ex22_02.hdf5')
savedfilename_best = os.path.join('cifar10', 'cifar10v3', 'checkpoints', 'Cifar10_Share_1_Inception_ex22_02_best.hdf5')
savedfilename_pre = os.path.join('cifar10', 'cifar10v3', 'checkpoints', 'Cifar10_Share_1_Inception_ex22_02_pre.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_accuracy', verbose=1, 
                          save_best_only=False, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('svhn_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('svhn_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

earlystopping = EarlyStoppingByAccVal(monitor='val_accuracy', value=0.9900, verbose=1)

def rand_scheduler(epoch, lr):
    rnd_lr = 10**(random.uniform(np.log10((1e-5)),np.log10((1e-1))))
#     if epoch < 30:
#         rnd_lr = 1e-2
#     else:    
#         rnd_lr = 1e-3
##     rnd_lr = lr
    print('random lr = ', rnd_lr)
    return rnd_lr

epochs = 40##!!!
lr = 1e-2
# decay = lr/epochs
# optimizer = Adam(lr=lr, decay=decay)
# optimizer = Adam(lr=lr)
optimizer = SGD(lr=lr)

# train on multiple-gpus
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of GPUs: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model_mul = get_adv_model()
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    # save initial models
    model_mul.save_weights(savedfilename)
    model_mul.save_weights(savedfilename_best)
    model_mul.save_weights(savedfilename_pre)

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

# result = model_mul.fit_generator(
#     generator = train_set, 
#     steps_per_epoch = step_size_train,
#     validation_data = valid_set,
#     validation_steps = step_size_valid,
#     shuffle=True,
#     epochs=epochs,
#     callbacks=[checkpointer],
# #     callbacks=[csv_logger, checkpointer, earlystopping],
# #     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
#     verbose=1) 

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of GPUs: 1
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
_____________________________________________________________________________

In [11]:
import pandas
import numpy as np
import os

d = {'id': [1, 2, 3, 4], 'pbest_value': [0, 0, 0, 0], 'pbest_file':['Cifar10_Share_1_Inception_ex22_02_best.hdf5',
                                                        'Cifar10_Share_2_Inception_ex22_02_best.hdf5',
                                                        'Cifar10_Share_3_Inception_ex22_02_best.hdf5',
                                                        'Cifar10_Share_4_Inception_ex22_02_best.hdf5'], 
                         'c_value': [0, 0, 0, 0], 'c_file':['Cifar10_Share_1_Inception_ex22_02.hdf5',
                                                             'Cifar10_Share_2_Inception_ex22_02.hdf5',
                                                             'Cifar10_Share_3_Inception_ex22_02.hdf5',
                                                             'Cifar10_Share_4_Inception_ex22_02.hdf5'], 
                         'pre_value': [0, 0, 0, 0], 'pre_file':['Cifar10_Share_1_Inception_ex22_02_pre.hdf5',
                                                             'Cifar10_Share_2_Inception_ex22_02_pre.hdf5',
                                                             'Cifar10_Share_3_Inception_ex22_02_pre.hdf5',
                                                             'Cifar10_Share_4_Inception_ex22_02_pre.hdf5'],
                         'training_flag':[0, 0, 0, 0]
    }
df = pandas.DataFrame(data=d)

In [12]:
df.to_csv(os.path.join('cifar10', 'cifar10v3', 'data_ex22_02.csv'))

In [13]:
data_file = os.path.join('cifar10', 'cifar10v3', 'data_ex22_02.csv')

def synch_read_data(data_file=''):
    while(True):
        try:
            df = pandas.read_csv(data_file, index_col=0)  
            break                     
        except:
            #waiting for 10s
            print("\n")
            for i in range(10,0,-1):
                print("re-read the file ....%2d" %i, end="\r", flush=True)
                time.sleep(1) 
    return df  

def synch_write_data(df,data_file=''):
    while(True):
        try:
            df.to_csv(data_file)  
            break                     
        except:
            #waiting for 10s
            print("\n")
            for i in range(10,0,-1):
                print("re-read the file ....%2d" %i, end="\r", flush=True)
                time.sleep(1) 
    return df  

def get_pbest_loc(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    pbest_value = row[1]
    file_name = row[2]
    return pbest_value, file_name

def set_pbest_loc(row, pbest_value):
    df = synch_read_data(data_file)
    df.loc[row, 'pbest_value'] = pbest_value
    synch_write_data(df,data_file)
    
def get_c_loc(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    c_value = row[3]
    file_name = row[4]
    return c_value, file_name

def set_c_loc(row, c_value):
    df = synch_read_data(data_file)
    df.loc[row, 'c_value'] = c_value
    synch_write_data(df,data_file)   

#    
def get_pre_loc(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    pre_value = row[5]
    file_name = row[6]
    return pre_value, file_name

def set_pre_loc(row, pre_value):
    df = synch_read_data(data_file)
    df.loc[row, 'pre_value'] = pre_value
    synch_write_data(df,data_file)
    
#training flag
def get_training_flag(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    training_flag = row[7]
    return training_flag

def set_training_flag(row, training_status):
    df = synch_read_data(data_file)
    df.loc[row, 'training_flag'] = training_status
    synch_write_data(df,data_file)

In [14]:
def find_distance(w1, w2):
    sqr_distance = 0
    
    w_np_1 = np.array(w1)
    w_fl_1 = w_np_1.flatten()
    w_np_2 = np.array(w2)
    w_fl_2 = w_np_2.flatten()
    
    for i in range(len(w_np_1)):
        x1_fl = w_fl_1[i].flatten()
        x2_fl = w_fl_2[i].flatten()

        tmp_dis = 0 
        for j in range(len(x1_fl)):
            tmp_dis = tmp_dis + (x1_fl[j]-x2_fl[j])**2

    #     print(tmp_dis)
        sqr_distance = sqr_distance + tmp_dis

    return sqr_distance**(1/2)  

In [16]:
from copy import deepcopy
import numpy as np
import keras
import math

#index of this pso
pso_index = 0

#number of neighbors (max=4)
num_neighbors = 4
#K coefficient
M = 1
u = 1

tmp_acc = 0
tmp_w = []
pbest_acc = 0
pbest_w = []

#accelerator coefficient
c1 = 0.5
c2 = 0.5
# w = 0.5

r1 = 0
r2 = 0

results_stack_accuracy = []
results_stack_val_accuracy = []
results_stack_loss = []
results_stack_val_loss = []

#threshold
# threshold = 0.97

# #iteration control
# i = 0
# iter_max = 40

warm_up = 0

for index in range(0,warm_up):
# while gbest_acc < threshold:
    #save previous weight
    model_mul.save_weights(savedfilename_pre)
    result = model_mul.fit_generator(
        generator = train_set, 
        steps_per_epoch = step_size_train,
        validation_data = valid_set,
        validation_steps = step_size_valid,
        shuffle=True,
        epochs=1,
        callbacks=[checkpointer,tf.keras.callbacks.LearningRateScheduler(rand_scheduler)],
    #     callbacks=[csv_logger, checkpointer, earlystopping],
    #     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
        verbose=1) 
    
    #save weights every iteration
#     model_mul.save_weights(savedfilename)   
    
    tmp_acc = result.history.get('val_accuracy')[-1]
    tmp_w = model_mul.get_weights()
    tmp_lr = result.history.get('lr')[-1]
    
    #save current location
    set_c_loc(pso_index,tmp_acc) 
    
    if tmp_acc > pbest_acc:
        pbest_acc = tmp_acc
        pbest_w = tmp_w
        #save person best location
        set_pbest_loc(pso_index,pbest_acc)  
        # save best model
        model_mul.save_weights(savedfilename_best) 
    
    results_stack_val_accuracy.append(result.history.get('val_accuracy')[-1])
    results_stack_accuracy.append(result.history.get('accuracy')[-1])
    results_stack_val_loss.append(result.history.get('val_loss')[-1])      
    results_stack_loss.append(result.history.get('loss')[-1])
    
#     i = i + 1
#     #exit after iteration gets max
#     if i > iter_max:
#         break

#    
# time synchronize
number_of_pso = 4
training_start_flag = 1
training_finish_flag = 0

#set initial training flag to start
set_training_flag(pso_index, training_start_flag)

for index in range(warm_up, epochs): 
# while i < iter_max:
    #start training 
    set_training_flag(pso_index, training_start_flag)
    print(get_training_flag(pso_index))
    
    #save previous weight
    model_mul.save_weights(savedfilename_pre) 
    
    result = model_mul.fit_generator(
        generator = train_set, 
        steps_per_epoch = step_size_train,
        validation_data = valid_set,
        validation_steps = step_size_valid,
        shuffle=True,
        epochs=1,
        callbacks=[checkpointer,tf.keras.callbacks.LearningRateScheduler(rand_scheduler)],
    #     callbacks=[csv_logger, checkpointer, earlystopping],
    #     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
        verbose=1) 
    
    #save weights every iteration
#     model_mul.save_weights(savedfilename)
    
    tmp_acc = result.history.get('val_accuracy')[-1]
    tmp_w = model_mul.get_weights()
    tmp_lr = result.history.get('lr')[-1]
    
    #save current location in scoreboard
    set_c_loc(pso_index,tmp_acc) 
    
    if tmp_acc > pbest_acc:
        pbest_acc = tmp_acc
        pbest_w = tmp_w
        #save person best location
        set_pbest_loc(pso_index,pbest_acc)  
        # save best model
        model_mul.save_weights(savedfilename_best)        

    #set training flag to finish
    set_training_flag(pso_index, training_finish_flag)  
    print(get_training_flag(pso_index))
        
    # check if all PSOs is ready (flag==1)
    while(True):
        tmp_flag = 0
        for flg_i in range(number_of_pso):
            print("flg_i", flg_i, "flag", get_training_flag(flg_i))
            if(get_training_flag(flg_i) == 1):
                tmp_flag = 1
        if(tmp_flag==1):
            #waiting for 60s
            print("\n")
            for i in range(60,0,-1):
                print("waiting for ....%2d" %i, end="\r", flush=True)
                time.sleep(1)    
        else:
            print("end of waiting")
            break                          
        
    r1 = random.uniform(0,1)
    r2 = random.uniform(0,1)
#     r3 = random.uniform(0,1)    
    
    #-----------nearest neighbor best--------------
    #get neighbor weights
    #1
    neighbor_c_acc_1, name_file_1 = get_c_loc(0)
    neighbor_c_acc_2, name_file_2 = get_c_loc(1)
    neighbor_c_acc_3, name_file_3 = get_c_loc(2)
    neighbor_c_acc_4, name_file_4 = get_c_loc(3)

    #get pre loc
    neighbor_pre_acc_1, name_pre_file_1 = get_pre_loc(0)
    neighbor_pre_acc_2, name_pre_file_2 = get_pre_loc(1)
    neighbor_pre_acc_3, name_pre_file_3 = get_pre_loc(2)
    neighbor_pre_acc_4, name_pre_file_4 = get_pre_loc(3)  
    
    #clone model for weights change
    model_clone = keras.models.clone_model(model_mul)
    
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_file_1))
    neighbor_w_1 = model_clone.get_weights() 
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_file_2))
    neighbor_w_2 = model_clone.get_weights()
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_file_3))
    neighbor_w_3 = model_clone.get_weights()
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_file_4))
    neighbor_w_4 = model_clone.get_weights()
    
    #clone model pre weights
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_pre_file_1))
    neighbor_pre_w_1 = model_clone.get_weights()     
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_pre_file_2))
    neighbor_pre_w_2 = model_clone.get_weights() 
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_pre_file_3))
    neighbor_pre_w_3 = model_clone.get_weights()    
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_pre_file_4))
    neighbor_pre_w_4 = model_clone.get_weights()     
    
    distance_1 = find_distance(neighbor_w_1,neighbor_w_2)
    distance_2 = find_distance(neighbor_w_1,neighbor_w_3)
    distance_3 = find_distance(neighbor_w_1,neighbor_w_4)
    
    #find the closest neighbor
    distances = list()
    distances.append((0,0))
    distances.append((1,distance_1))
    distances.append((2,distance_2))
    distances.append((3,distance_3))

    print('distances unsorted', distances)
    
    distances.sort(key=lambda tup: tup[1])
    print('distances ', distances)
    
    neighbors_idx = list()
    for i in range(num_neighbors):
        neighbors_idx.append(distances[i][0])        
    
    print('neighbors ids ', neighbors_idx)
    
    #get neighbor bests from the list
    neighbor_bests = list()
    #remove first element (self distance)
#     neighbors_idx.pop(0)
    
    for i in range(len(neighbors_idx)):
        neighbor_best_tmp, name_file_neighbor_best_tmp = get_pbest_loc(neighbors_idx[i])
        neighbor_bests.append((neighbors_idx[i],neighbor_best_tmp))
        print('neighbor_idx ', neighbors_idx[i])
        print('neighbor_best_tmp ', neighbor_best_tmp)
    
    # keep unsorted list of neighbor
    neighbor_tmp = deepcopy(neighbor_bests)
    
    # sort the list for maximum accuracy   
    neighbor_bests.sort(key=lambda tup: tup[1], reverse=True)
    print('neighbor best ', neighbor_bests)
    #
    neighbor_best_value, name_file_neighbor_best = get_pbest_loc(neighbor_bests[0][0])
    print('name_file_neighbor_best ', name_file_neighbor_best)
    
    model_clone.load_weights(os.path.join('cifar10', 'cifar10v3', 'checkpoints', name_file_neighbor_best))
    neighbor_best_w = model_clone.get_weights()  
    #---------- end nearest neighbor best ----------
    
    #---------- cucker -----------------------------
    particle_w_i = neighbor_w_1
    sum_particle_tmp = 0
    
    #remove the fist (self)
    neighbor_tmp.pop(0)
    
    for j in range(len(neighbor_tmp)):
        if neighbor_tmp[j][0]==1:
            particle_w_j = neighbor_w_2
            particle_w_pre_j = neighbor_pre_w_2
            distance_ij = distance_1
            u = 0.2
        elif neighbor_tmp[j][0]==2:    
            particle_w_j = neighbor_w_3
            particle_w_pre_j = neighbor_pre_w_3
            distance_ij = distance_2
            u = 0.2
        elif neighbor_tmp[j][0]==3:    
            particle_w_j = neighbor_w_4
            particle_w_pre_j = neighbor_pre_w_4
            distance_ij = distance_3
            u = 1.7
            
        print('u ', u)
        print('distance_ij ', distance_ij)
        #sum(K/(1+distance)*(particle_w_j-particle_w_i)
        sum_particle_tmp -= M*u/(1+distance_ij)*(np.array(particle_w_pre_j)-np.array(particle_w_j)) 
        
    #---------- end cucker -------------------------
    
    #---------- pbest ------------------------------
    
    #---------- end pbest --------------------------

    #update networks' weights
    #     w = c1*r1*(np.array(pbest_w)-np.array(tmp_w))+c2*r2*(np.array(gbest_w)-np.array(tmp_w))
    #     w = r1*np.array(pbest_w)+r2*np.array(tmp_w)+r3*np.array(gbest_w)
    #     w = np.array(tmp_w)+tmp_lr*(c1*r1*(np.array(pbest_w)-np.array(tmp_w))+c2*r2*(np.array(gbest_w)-np.array(tmp_w)))
#     final_weight = np.array(tmp_w)+sum_particle_tmp+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))

#     final_weight = np.array(tmp_w)+sum_particle_tmp+c1*r1*(np.array(pbest_w)-np.array(tmp_w))+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
    final_weight = np.array(tmp_w)+sum_particle_tmp+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
#     final_weight = sum_particle_tmp+np.array(neighbor_best_w)
#     final_weight = np.array(tmp_w)+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
#     final_weight = np.array(tmp_w)+sum_particle_tmp

    model_mul.set_weights(final_weight)
    
    print('After ---> epoch=', index, ' r1=',r1, ' r2=',r2, ' current acc=', tmp_acc, ' local best=', pbest_acc, 
          ' neighbor index=', neighbor_bests[0][0], ' neighbor best=', neighbor_best_value)  
    
    results_stack_val_accuracy.append(result.history.get('val_accuracy')[-1])
    results_stack_accuracy.append(result.history.get('accuracy')[-1])
    results_stack_val_loss.append(result.history.get('val_loss')[-1])      
    results_stack_loss.append(result.history.get('loss')[-1])
    
#     i = i + 1
        
print(results_stack_val_accuracy)

1
Instructions for updating:
Please use Model.fit, which supports generators.
random lr =  0.0006450243061671104
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
1563/1563 [==============================] - ETA: 0s - loss: 1.8706 - accuracy: 0.3713
Epoch 00001: saving model to cifar10/cifar10v3/checkpoints/Cifar10_Share_1_Inception_ex22_02.hdf5
1563/1563 [==============================] - 945s 604ms/step - loss: 1.8706 - accuracy: 0.3713 - val_loss: 1.1373 - val_accuracy: 0.6942
0
flg_i 0 flag 0
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_

u  0.2
distance_ij  31.53839089054396
u  0.2
distance_ij  31.87623150641179
u  1.7
distance_ij  41.44508646068291
After ---> epoch= 4  r1= 0.15963419630321773  r2= 0.6772601965461791  current acc= 0.9581999778747559  local best= 0.9581999778747559  neighbor index= 1  neighbor best= 0.9668999910354614
1
random lr =  0.004246842362891913
1563/1563 [==============================] - ETA: 0s - loss: 0.2575 - accuracy: 0.9217
Epoch 00001: saving model to cifar10/cifar10v3/checkpoints/Cifar10_Share_1_Inception_ex22_02.hdf5
1563/1563 [==============================] - 941s 602ms/step - loss: 0.2575 - accuracy: 0.9217 - val_loss: 0.1169 - val_accuracy: 0.9621
0
flg_i 0 flag 0
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 0
end of waiting
distances unsorted [(0, 0), (1, 21.24986396049563), (2, 21.689973377619495), (3, 30.04044301492213)]
distances  [(0, 0), (1, 21.24986396049563), (2, 21.689973377619495), (3, 30.04044301492213)]
neighbors ids  [0, 1, 2, 3]
neighbor_idx  0
neighbor_best_tmp  0.9621

u  0.2
distance_ij  4.629315608901521
u  0.2
distance_ij  5.998830419401524
u  1.7
distance_ij  23.540845385323777
After ---> epoch= 10  r1= 0.5878192212154613  r2= 0.14777583186741927  current acc= 0.9725000262260437  local best= 0.9725000262260437  neighbor index= 0  neighbor best= 0.9725000262260436
1
random lr =  0.003349876959363634
1563/1563 [==============================] - ETA: 0s - loss: 0.1416 - accuracy: 0.9523
Epoch 00001: saving model to cifar10/cifar10v3/checkpoints/Cifar10_Share_1_Inception_ex22_02.hdf5
1563/1563 [==============================] - 948s 606ms/step - loss: 0.1416 - accuracy: 0.9523 - val_loss: 0.0847 - val_accuracy: 0.9713
0
flg_i 0 flag 0
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 0
end of waiting
distances unsorted [(0, 0), (1, 9.612178979995514), (2, 9.685014280981676), (3, 14.2500613758232)]
distances  [(0, 0), (1, 9.612178979995514), (2, 9.685014280981676), (3, 14.2500613758232)]
neighbors ids  [0, 1, 2, 3]
neighbor_idx  0
neighbor_best_tmp  0.972500

u  0.2
distance_ij  7.566441099237365
u  0.2
distance_ij  8.229950160662893
u  1.7
distance_ij  20.282121288876915
After ---> epoch= 16  r1= 0.026858068138373326  r2= 0.7093352298505248  current acc= 0.9739000201225281  local best= 0.9750999808311462  neighbor index= 0  neighbor best= 0.9750999808311462
1
random lr =  0.0004731584510260895
1563/1563 [==============================] - ETA: 0s - loss: 0.0849 - accuracy: 0.9716
Epoch 00001: saving model to cifar10/cifar10v3/checkpoints/Cifar10_Share_1_Inception_ex22_02.hdf5
1563/1563 [==============================] - 937s 600ms/step - loss: 0.0849 - accuracy: 0.9716 - val_loss: 0.0752 - val_accuracy: 0.9758
0
flg_i 0 flag 0
flg_i 1 flag 1
flg_i 2 flag 1
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 0
end of waiting
distances unsorted [(0, 0), (1, 9.769780625239449), (2, 11.882091557971773), (3, 19.68450626667247)]
distances  [(0, 0), (1, 9.769780625239449), (2, 11.882091557971773), (3, 19.68450626667247)]

u  0.2
distance_ij  11.59336244091297
u  0.2
distance_ij  12.759486395663897
u  1.7
distance_ij  35.51436541984239
After ---> epoch= 22  r1= 0.8941994867069547  r2= 0.3491668336488687  current acc= 0.9589999914169312  local best= 0.9771000146865845  neighbor index= 0  neighbor best= 0.9771000146865844
1
random lr =  0.029854035489638366
1563/1563 [==============================] - ETA: 0s - loss: 0.1750 - accuracy: 0.9410
Epoch 00001: saving model to cifar10/cifar10v3/checkpoints/Cifar10_Share_1_Inception_ex22_02.hdf5
1563/1563 [==============================] - 944s 604ms/step - loss: 0.1750 - accuracy: 0.9410 - val_loss: 0.1535 - val_accuracy: 0.9514
0
flg_i 0 flag 0
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 0
end of waiting
distances unsorted [(0, 0), (1, 16.789476929296242), (2, 15.519119077601935), (3, 21.310176321128758)]
distances  [(0, 0), (2, 15.519119077601935), (1, 16.789476929296242), (3, 21.310176321128758)]
neighbors ids  [0, 2, 1, 3]
neighbor_idx  0
neighbor_best_tmp  0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - ETA: 0s - loss: 0.0713 - accuracy: 0.9767
Epoch 00001: saving model to cifar10/cifar10v3/checkpoints/Cifar10_Share_1_Inception_ex22_02.hdf5
1563/1563 [==============================] - 947s 606ms/step - loss: 0.0713 - accuracy: 0.9767 - val_loss: 0.0737 - val_accuracy: 0.9770
0
flg_i 0 flag 0
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 0
end of waiting
distances unsorted [(0, 0), (1, 8.623768980795308), (2, 4.592509168954635), (3, 9.604848206388576)]
distances  [(0, 0), (2, 4.592509168954635), (1, 8.623768980795308), (3, 9.604848206388576)]
neighbors ids  [0, 2, 1, 3]
neighbor_idx  0
neighbor_best_tmp  0.9771000146865844
neighbor_idx  2
neighbor_best_tmp  0.9764999747276306
neighbor_idx  1
neighbor_best_tmp  0.9769999980926514
neighbor_idx  3
neighbor_best_tmp  0.97680002450943
neighbor best  [(0, 0.9771000146865844), (1, 0.9769999980926514), (3, 0.97680002450943), (2, 0.9764999747276306)]
name_file_neighbor_best  Cifar10_Share_1_Inception_ex

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - ETA: 0s - loss: 0.0479 - accuracy: 0.9837
Epoch 00001: saving model to cifar10/cifar10v3/checkpoints/Cifar10_Share_1_Inception_ex22_02.hdf5
1563/1563 [==============================] - 943s 603ms/step - loss: 0.0479 - accuracy: 0.9837 - val_loss: 0.0715 - val_accuracy: 0.9791
0
flg_i 0 flag 0
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 1


flg_i 0 flag 0.. 1
flg_i 1 flag 0
flg_i 2 flag 0
flg_i 3 flag 0
end of waiting
distances unsorted [(0, 0), (1, 4.542835675696545), (2, 5.107584878368981), (3, 12.267587464769505)]
distances  [(0, 0), (1, 4.542835675696545), (2, 5.107584878368981), (3, 12.267587464769505)]
neighbors ids  [0, 1, 2, 3]
neighbor_idx  0
neighbor_best_tmp  0.9790999889373779
neighbor_idx  1
neighbor_best_tmp  0.9796000123023988
neighbor_idx  2
neighbor_best_tmp  0.977400004863739
neighbor_idx  3
neighbor_best_tmp  0.9782999753952026
neighbor best  [(1, 0.9796000123023988), (0, 0.9790999889373779), (3, 0.9782999753952026), (2, 0.9

In [17]:
print(results_stack_val_accuracy)

[0.6941999793052673, 0.8855999708175659, 0.9434000253677368, 0.9524999856948853, 0.9581999778747559, 0.9621000289916992, 0.9675999879837036, 0.9689000248908997, 0.9593999981880188, 0.9703999757766724, 0.9725000262260437, 0.9713000059127808, 0.9623000025749207, 0.9717000126838684, 0.9750999808311462, 0.9670000076293945, 0.9739000201225281, 0.9757999777793884, 0.9750000238418579, 0.9750999808311462, 0.9757000207901001, 0.9771000146865845, 0.9589999914169312, 0.9513999819755554, 0.9692999720573425, 0.9731000065803528, 0.9746000170707703, 0.9757000207901001, 0.9036999940872192, 0.968999981880188, 0.9768000245094299, 0.9769999980926514, 0.9771999716758728, 0.9778000116348267, 0.9786999821662903, 0.9782000184059143, 0.9304999709129333, 0.9758999943733215, 0.9790999889373779, 0.9790999889373779]


In [18]:
print(results_stack_val_accuracy)

[0.6941999793052673, 0.8855999708175659, 0.9434000253677368, 0.9524999856948853, 0.9581999778747559, 0.9621000289916992, 0.9675999879837036, 0.9689000248908997, 0.9593999981880188, 0.9703999757766724, 0.9725000262260437, 0.9713000059127808, 0.9623000025749207, 0.9717000126838684, 0.9750999808311462, 0.9670000076293945, 0.9739000201225281, 0.9757999777793884, 0.9750000238418579, 0.9750999808311462, 0.9757000207901001, 0.9771000146865845, 0.9589999914169312, 0.9513999819755554, 0.9692999720573425, 0.9731000065803528, 0.9746000170707703, 0.9757000207901001, 0.9036999940872192, 0.968999981880188, 0.9768000245094299, 0.9769999980926514, 0.9771999716758728, 0.9778000116348267, 0.9786999821662903, 0.9782000184059143, 0.9304999709129333, 0.9758999943733215, 0.9790999889373779, 0.9790999889373779]


In [19]:
print(results_stack_val_loss)

[1.1372584104537964, 0.4404491186141968, 0.17097197473049164, 0.1411975920200348, 0.16878946125507355, 0.11687790602445602, 0.09897471219301224, 0.09823878109455109, 0.1188531368970871, 0.08871538192033768, 0.08449757844209671, 0.08473727107048035, 0.10772871226072311, 0.08351626992225647, 0.07656248658895493, 0.10432229191064835, 0.08219543099403381, 0.07520879060029984, 0.07708664238452911, 0.07739999145269394, 0.07660680264234543, 0.07443089038133621, 0.12739796936511993, 0.15347349643707275, 0.0970914289355278, 0.0852443054318428, 0.0767538771033287, 0.07498301565647125, 0.31512510776519775, 0.10020239651203156, 0.07574762403964996, 0.07366904616355896, 0.07233405858278275, 0.0703732818365097, 0.06835057586431503, 0.06871092319488525, 0.23371060192584991, 0.07974554598331451, 0.07038229703903198, 0.07145204395055771]


In [20]:
print(results_stack_loss)

[1.8706386089324951, 0.9649229645729065, 0.4593749940395355, 0.3043317496776581, 0.3182665705680847, 0.2575303614139557, 0.19917386770248413, 0.18913178145885468, 0.22340010106563568, 0.14900417625904083, 0.13807624578475952, 0.14162763953208923, 0.18335744738578796, 0.11193860322237015, 0.10380949825048447, 0.13455986976623535, 0.09674745798110962, 0.084928497672081, 0.08524511754512787, 0.08367593586444855, 0.06733880937099457, 0.06752101331949234, 0.13639715313911438, 0.17499035596847534, 0.11224040389060974, 0.08264434337615967, 0.06945127248764038, 0.062029652297496796, 0.3011375665664673, 0.12266816198825836, 0.07744115591049194, 0.07131702452898026, 0.06536724418401718, 0.06122374162077904, 0.053202949464321136, 0.05321474000811577, 0.12421479821205139, 0.06167980283498764, 0.048221684992313385, 0.04789896309375763]


In [ ]:
result.history.get('val_accuracy')

In [ ]:
result.history.get('val_loss')

In [ ]:
result.history.get('loss')

In [ ]:
# check if all PSOs is ready (flag==1)
while(True):
    try:
        tmp_flag = 0
        for flg_i in range(4):
            print("flg_i", flg_i, "flag", get_training_flag(flg_i))
            for i in range(1,0,-1):
                print("waiting for ....%2d" %i, end="\r", flush=True)                
            if(get_training_flag(flg_i) == 1):
                tmp_flag = 1
        if(tmp_flag==1):
            #waiting for 60s
            print("\n")
            for i in range(1,0,-1):
                print("waiting for ....%2d" %i, end="\r", flush=True)
                time.sleep(1)        
        else:
            print("end of waiting")
            break 
    except:
        #waiting for 10s
        print("\n")
        for i in range(10,0,-1):
            print("exception ....%2d" %i, end="\r", flush=True)
            time.sleep(1)

In [ ]:
time.sleep(1)

In [ ]:
try:
    print("hello")
except:
    print("An exception occurred")

In [ ]:
model_mul.load_weights(os.path.join('checkpoints', 'Cifar10_EfficientNetB7_299_STD.hdf5'))

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

savedfilename = os.path.join('checkpoints', 'Cifar10_EfficientNetB7_299_STD_L2.hdf5')
checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

epochs = 15##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = model_mul.fit_generator(
    generator = train_set_1, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
model_mul.load_weights(os.path.join('checkpoints', 'Cifar10_EfficientNetB7_299_STD_L2.hdf5'))

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

savedfilename = os.path.join('checkpoints', 'Cifar10_EfficientNetB7_299_STD_L3.hdf5')
checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

epochs = 15##!!!
lr = 1e-6
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = model_mul.fit_generator(
    generator = train_set_1, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
# #Using multiple models if more than 1 GPU
# NUM_GPU = 4
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_1, gpus=NUM_GPU)

model_mul.load_weights(os.path.join('checkpoints', 'Cifar10_EfficientNetB7_299_STD_L3.hdf5'))

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import time, os
from math import ceil

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 40

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cifar10_EfficientB7_299_STD_2708.csv')
results.head()

In [ ]:
cp Cifar10_Eff_B5_345_1511_v1.csv /home/bribeiro/Phong/Nat19/Cifar10_Eff_B5_345_1511_v1.csv

In [ ]:
np.save(os.path.join('pred_npy','Cifar10_EfficientB7_299_STD_L3.npy'), predict1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np

batch_size = 72

#Crop-Official Test
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator(batches, crop_length):
    """Generate random crops from the image batches"""
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

test_datagen_crop = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

testing_set_crop = test_datagen_crop.flow_from_directory('test_resized_345',
                                                 target_size = (370, 370),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="training"
                                              )
#customized generator
test_crops = crop_generator(testing_set_crop, 345)

step_size_test_crop = ceil(testing_set_crop.n/testing_set_crop.batch_size)

tta_steps = 4
# predictions = []

# import tensorflow as tf
# with tf.device('/gpu:0'):
for i in range(tta_steps):
    print(i)
    testing_set_crop.reset()
    if NUM_GPU != 1:
        preds=model_mul.predict_generator(test_crops, 
                                           steps = step_size_test_crop,
#                                            max_queue_size=16,
#                                                use_multiprocessing=True,
#                                            workers=1,
                                           verbose=1)    
#     else:
#         preds=model.predict_generator(test_crops, 
#                                            steps = step_size_test_crop,
#                                            max_queue_size=16,
# #                                                use_multiprocessing=True,
#                                            workers=1,
#                                            verbose=1)  
#     preds=model_2.predict_generator(test_crops,steps = step_size_test_crop,verbose=1)  
    predictions.append(preds)

mean_pred = np.mean(predictions, axis=0)

predicted_class_indices_mean=np.argmax(mean_pred,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=testing_set_crop.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_Eff_B5_345_STD_tta_7.csv')
results.head(10)

In [ ]:
cp Cifar10_Eff_B5_345_STD_tta_7.csv /home/bribeiro/Phong/Nat19/Cifar10_Eff_B5_345_STD_tta_7.csv

In [ ]:
np.save(os.path.join('pred_npy','Cifar10_Eff_B5_345_L2_TTA3.npy'), mean_pred)

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

savedfilename = os.path.join('checkpoints', 'Cifar100_Eff_B7_299_STD_L3.hdf5')
checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

epochs = 15##!!!
lr = 1e-6
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
# #Using multiple models if more than 1 GPU
# NUM_GPU = 4
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_1, gpus=NUM_GPU)

model_mul.load_weights(os.path.join('checkpoints', 'Cifar100_Eff_B7_299_STD_L3.hdf5'))

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import time, os
from math import ceil

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 36

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cifar100_Eff_B0_299_1108_v1.csv')
results.head()

In [ ]:
np.save(os.path.join('pred_npy','Cifar100_Eff_B7_299_STD_L3.npy'), predict1)